# temp BA-288 Список оформленных заказов

## magic

In [22]:
# magic    
get_ipython().run_line_magic('matplotlib', 'inline')
# run files
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Import_libs.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Connecting_.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/my_script.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Date_gen.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/xx.py')

# print
###########################
# print(
# df_test_xx.shape, '\n'
# ,abs_path, '\n' # abs_path - путь до шары (видна всем езерам - '/srv/jhub/share/')
# ,path_OUT_dbDATA, '\n'
# ,date_start, date_end, '\n'
# )
############################## End ##############################


In [24]:
# выводим спарсенные данные с жиры описание и тд....
task_link = '''
https://jira.e2e4.ru/browse/BA-\
458
'''
tasks(loginAtlassianjhubAdmin, passwordAtlassianjhubAdmin, task_link)

Данные с jira 
(подтягиваются автоматически и обнавляются при запуске)


HTML(value="<a href='\nhttps://jira.e2e4.ru/browse/BA-458\n'>\nhttps://jira.e2e4.ru/browse/BA-458\n</a>")

Бизнес аналитика / BA-458 

        Разовая выгрузка заказов по дате оформления  


        Описание: 
        
Для сверки вэб-аналитики необходима единоразовая выгрузка заказов за последние 2 недели (по дате оформления позиции).
Принимаются к учету только заказы с ID площадки 7832, 1246, 3252. Учитываются те позиции и заказы, которые есть на момент выгрузки (без логов), независимо от статуса заказа (включая удаленные).
Форма вывода (1 строка, 1 позиция):
ID заказа| Дата и время с секундами оформления позиции| Дата  и время с секундами оформления заказа| Кто оформил заказ
фильтр по дате оф заказа 
последние 2 недели 

        


## Качнуть за какую то опредлеленную дату

In [25]:
# #  Качнуть за какую то опредлеленную дату
# date_start = '01.10.2021'
# date_end = '31.10.2021'

# path_OUT_dbDATA + folder_out + file_name + fr' { date_start } { date_end }.csv'

## SQL

In [26]:
# DATE_START_replce date_start
# DATE_END_replce date_end

sql_ = '''     

    /* 
    https://jira.e2e4.ru/browse/BA-458
    */

    with params as (
        select
             ('31.01.2023' || ' 00:00:00')::timestamp frm, -- 6 556
             ('13.02.2023' || ' 23:59:59')::timestamp til
    --   ( 'DATE_START_replce' || ' 00:00:00')::timestamp    frm 
    --   ,( 'DATE_END_replce' || ' 23:59:59')::timestamp    til 
    )
    -- select * from params

    , wares as  (
        select
            -- a.i AS i_log,
            a.i,
            tm_placed,
            a.in_account_date,
            a.user_login AS order_source,
            ai.id_platform,

            a.id AS id_order,        
            null::INT AS id_order_item,
            a.id_customer AS id_customer,
            a.firm AS order_type,
            a.storage_id AS id_storage,
            'Товар' AS TYPE,
            rp.id AS id,
            rp.name AS name,
            g.name AS category,
            b.name AS brand,
            s.price AS price,
            round(s.cost) AS cost,
            round(s.sprice) AS sprice,
            s.date_sold AS date_sold,
            s.sold AS state, 
            COALESCE(r.date, checks.created_at) AS realization_date,
            ROUND(s.price * (COALESCE(rnds.value, cnds.value, 20) / (100 + COALESCE(rnds.value, cnds.value, 20))), 2) AS nds,
            c.name AS supplier,
            u.id AS manager
            --Состояние сборки: (oc.collect) 0 - сборка не заказана, 1 - собрать, 2 - собрана, 3 - разобрать, 4 - разобрана, 5 - продана
            ,case when (oc.collect = 0) or (oc.collect is null) then null else 'c' end "сборка/комплект" 
            ,case when (oc.collect = 0) or (oc.collect is null) then null else oc.collectcount::text end "Кол-во сборок" --oc.collectcount - Количество собранных компьютеров в составе заказа
    --        ,a.in_account_date in_account_date
            ,a.state item_state --Состояниt позиций
            ,a.del adel --Позиция удалена --лучше orders.item_state('DELETED')

        FROM params _
        CROSS JOIN opentech.accounts a
        LEFT JOIN opentech.account_item ai ON ai.account_id = a.id
        JOIN opentech.reference_price rp ON rp.id = a.reference_id -- rp.id = s.reference_id
        LEFT join opentech.store s ON a.store_id = s.id
        LEFT join opentech.order_client oc ON a.id = oc.account_id
        LEFT JOIN opentech.users u ON lower(u.login) = lower(a.user_login)
        LEFT JOIN orders.realization_items_order_items_bindings bi ON bi.id_order_item = a.i
        LEFT JOIN orders.realization_items i ON i.id = bi.id_realization_item
        LEFT JOIN orders.realizations r ON r.id = i.id_realization
        LEFT JOIN payments.order_items_locked_by_check_items il ON il.id_order_item = a.i
        LEFT JOIN payments.check_items ci ON ci.id = il.id_check_item_fullpaid
        LEFT JOIN payments.checks checks ON checks.id = ci.id_check
        LEFT JOIN public.nds rnds ON rnds.id = i.id_nds
        LEFT JOIN public.nds cnds ON cnds.code = ci.vat
        LEFT JOIN opentech.section_new sn ON sn.id = rp.section_sootv
        LEFT JOIN catalog.plain_section_list g ON g.id = sn.goodid
        LEFT JOIN catalog.brands b ON b.id = rp.brand_id
        LEFT JOIN opentech.supplyitems si ON si.id = s.supplyitemid
        LEFT JOIN opentech.supplies su ON su.id = si.supplyid
        LEFT JOIN customers.suppliers sup ON sup.id_customer = su.supplierid
        LEFT JOIN opentech.customers c ON c.id = COALESCE(sup.id_customer_alpha, sup.id_customer)
        WHERE 
            a.state != orders.item_state('BUNDLED') -- WHEN 'BUNDLED' THEN RETURN 27;                  --Комплект
    --        AND s.date_sold BETWEEN _.frm AND _.til
    --        AND a.in_account_date BETWEEN _.frm AND _.til
            AND tm_placed BETWEEN _.frm AND _.til
            and ai.id_platform in (7832, 1246, 3252)

            )

    , bundles AS  (
        SELECT
            -- a.i AS i_log,
            a2.i,
            tm_placed,
            a1.in_account_date,
            a2.user_login AS order_source,
            ai.id_platform,

            a2.id AS id_order,          /* id заказа комплекта */
            a1.id AS id_order_item,     /* id заказа комплектующей */
            a2.id_customer AS id_customer,
            a2.firm AS order_type,
            a2.storage_id AS id_storage,
            'Комплектующая' AS TYPE,
            rp.id AS id,
            rp.name AS name,
            g.name AS category,
            b.name AS brand,
            s1.price AS price,
            round(s1.cost) AS cost,
            round(s1.sprice) AS sprice,
            s2.date_sold AS date_sold,
            s1.sold AS state, 
            COALESCE(r.date, checks.created_at) AS realization_date,
            ROUND(s1.price * (COALESCE(rnds.value, cnds.value, 20) / (100 + COALESCE(rnds.value, cnds.value, 20))), 2) AS nds,
            c.name AS supplier,
            u.id AS manager
            ,'к' "сборка/комплект" 
            ,null
    --        ,a2.in_account_date in_account_date
            ,a1.state item_state
            ,a1.del adel

        FROM params _
        CROSS JOIN opentech.accounts a1                            /* Складская позиция комплектующей */
        LEFT JOIN opentech.store s1 ON a1.store_id = s1.id        /* Заказ комплектующей */
        LEFT JOIN store.bundled_items bu ON bu.id_store_item = s1.id
        LEFT JOIN opentech.store s2 ON s2.id = bu.id_bundle          /* Складская позиция комплекта */
        LEFT JOIN opentech.accounts a2 ON a2.store_id = s2.id        /* Заказ комплекта */
        LEFT JOIN opentech.account_item ai ON ai.account_id = a1.id

        /* НДС комплектующей */
        LEFT JOIN orders.realization_items_order_items_bindings bi1 ON bi1.id_order_item = a1.i
        LEFT JOIN orders.realization_items i1 ON i1.id = bi1.id_realization_item
        LEFT JOIN payments.order_items_locked_by_check_items il1 ON il1.id_order_item = a1.i
        LEFT JOIN payments.check_items ci1 ON ci1.id = il1.id_check_item_fullpaid
        LEFT JOIN public.nds rnds ON rnds.id = i1.id_nds
        LEFT JOIN public.nds cnds ON cnds.code = ci1.vat
        /* Комплектующая */
        JOIN opentech.reference_price rp ON rp.id = a1.reference_id --rp.id = s1.reference_id
        LEFT JOIN opentech.section_new sn ON sn.id = rp.section_sootv
        LEFT JOIN catalog.plain_section_list g ON g.id = sn.goodid
        LEFT JOIN catalog.brands b ON b.id = rp.brand_id
        /* Комплект */    
        LEFT JOIN opentech.users u ON lower(u.login) = lower(a2.user_login)
        LEFT JOIN orders.realization_items_order_items_bindings bi ON bi.id_order_item = a2.i
        LEFT JOIN orders.realization_items i ON i.id = bi.id_realization_item
        LEFT JOIN orders.realizations r ON r.id = i.id_realization
        LEFT JOIN payments.order_items_locked_by_check_items il ON il.id_order_item = a2.i
        LEFT JOIN payments.check_items ci ON ci.id = il.id_check_item_fullpaid
        LEFT JOIN payments.checks checks ON checks.id = ci.id_check
        LEFT JOIN opentech.supplyitems si ON si.id = s2.supplyitemid
        LEFT JOIN opentech.supplies su ON su.id = si.supplyid
        LEFT JOIN customers.suppliers sup ON sup.id_customer = su.supplierid
        LEFT JOIN opentech.customers c ON c.id = COALESCE(sup.id_customer_alpha, sup.id_customer)
        WHERE 
            a1.state = orders.item_state('BUNDLED')
    --        AND s2.date_sold BETWEEN _.frm AND _.til
    --        AND a1.in_account_date BETWEEN _.frm AND _.til
            AND ai.tm_placed BETWEEN _.frm AND _.til
            and ai.id_platform in (7832, 1246, 3252)

    )

    , services AS  (
        SELECT
            -- a.i AS i_log,
            a.i,
            tm_placed,
            a.in_account_date,
            a.user_login AS order_source,
            ai.id_platform,

            a.id AS id_order,
            null::INT AS id_order_item,
            a.id_customer AS id_customer,
            a.firm AS order_type,
            a.storage_id AS id_storage,
            'Услуга' AS TYPE,
            t.id AS id,
            t.name AS name,
            c.name AS category,
            NULL::TEXT AS brand,
            a.price AS price,
            NULL::int AS cost,
            NULL::int AS sprice,
            os.done AS date_sold,
            store.item_sold('SHIPPED') AS state,
            COALESCE(r.date, checks.created_at) AS realization_date,
            ROUND(a.price * (COALESCE(rnds.value, cnds.value, 20) / (100 + COALESCE(rnds.value, cnds.value, 20))), 2) AS nds,
            NULL::TEXT AS supplier,
            u.id AS manager
            ,null
            ,null
    --        ,a.in_account_date in_account_date
            ,os.state item_state
            ,a.del adel

        FROM params _
        CROSS JOIN opentech.accounts a
            LEFT JOIN opentech.account_item ai ON ai.account_id = a.id

        LEFT JOIN opentech.orderservices os ON a.i = os.orderitemid
        LEFT JOIN opentech.users u ON lower(u.login) = lower(a.user_login)
        LEFT JOIN orders.realization_items_order_items_bindings bi ON bi.id_order_item = a.i
        LEFT JOIN orders.realization_items i ON i.id = bi.id_realization_item
        LEFT JOIN orders.realizations r ON r.id = i.id_realization
        LEFT JOIN payments.order_items_locked_by_check_items il ON il.id_order_item = a.i
        LEFT JOIN payments.check_items ci ON ci.id = il.id_check_item_fullpaid
        LEFT JOIN payments.checks checks ON checks.id = ci.id_check
        LEFT JOIN public.nds rnds ON rnds.id = i.id_nds
        LEFT JOIN public.nds cnds ON cnds.code = ci.vat
        JOIN service.types t ON t.id = os.type
        JOIN service.categories c ON c.id = t.id_category
        WHERE 
    --    	os.state = service.state('SOLD')
    --        os.done BETWEEN _.frm AND _.til
    --        a.in_account_date BETWEEN _.frm AND _.til
    --        AND 
            tm_placed BETWEEN _.frm AND _.til
            and ai.id_platform in (7832, 1246, 3252)

    )

    , deliveries AS  (
        SELECT
            -- a.i AS i_log,
            a.i,
            tm_placed,
            a.in_account_date,
            a.user_login AS order_source,
            ai.id_platform,

            a.id AS id_order,
            null::INT AS id_order_item,
            a.id_customer AS id_customer,
            a.firm AS order_type,
            a.storage_id AS id_storage,
            'Доставка' AS TYPE,
            d.id AS id,
            a.correct_name AS name,
            t.name AS category,
            NULL::text AS brand,
            a.price AS price,
            NULL::int AS cost,
            NULL::int AS sprice,
            d.date AS date_sold,
            store.item_sold('SHIPPED') AS state,
            COALESCE(r.date, checks.created_at) AS realization_date,
            ROUND(a.price * (COALESCE(rnds.value, cnds.value, 20) / (100 + COALESCE(rnds.value, cnds.value, 20))), 2) AS nds,
            NULL::text AS supplier,
            u.id AS manager
            ,null
            ,null
    --        ,a.in_account_date in_account_date
            ,d.state item_state
            ,a.del adel

        FROM params _
        CROSS JOIN opentech.accounts a
            LEFT JOIN opentech.account_item ai ON ai.account_id = a.id

        LEFT JOIN logistics.delivery_items di ON a.i = di.id_order_item AND a.correct_name ILIKE 'Доставка%'
        LEFT JOIN logistics.deliveries d  ON di.id_delivery = d.id
        LEFT JOIN logistics.delivery_types t ON t.id = d.type
        LEFT JOIN opentech.users u ON lower(u.login) = lower(a.user_login)
        LEFT JOIN orders.realization_items_order_items_bindings bi ON bi.id_order_item = a.i
        LEFT JOIN orders.realization_items i ON i.id = bi.id_realization_item
        LEFT JOIN orders.realizations r ON r.id = i.id_realization
        LEFT JOIN payments.order_items_locked_by_check_items il ON il.id_order_item = a.i
        LEFT JOIN payments.check_items ci ON ci.id = il.id_check_item_fullpaid
        LEFT JOIN payments.checks checks ON checks.id = ci.id_check
        LEFT JOIN public.nds rnds ON rnds.id = i.id_nds
        LEFT JOIN public.nds cnds ON cnds.code = ci.vat
        WHERE 
    --    	and d.state = logistics.delivery_state('CLOSED')
    --        d.date BETWEEN _.frm AND _.til
    --        AND 
            tm_placed BETWEEN _.frm AND _.til
            and ai.id_platform in (7832, 1246, 3252)

    )

    , giftcards AS  (
        SELECT
            -- a.i AS i_log,
            a.i,
            tm_placed,
            a.in_account_date,
            a.user_login AS order_source,
            ai.id_platform,

            a.id AS id_order,
            null::INT AS id_order_item,
            a.id_customer AS id_customer,
            a.firm AS order_type,
            a.storage_id AS id_storage,
            'Подарочная карта' AS TYPE,
            1 AS id,
            a.correct_name AS name,
            'Подарочная карта' AS category,
            NULL::text AS brand,
            a.price AS price,
            NULL::int AS cost,
            NULL::int AS sprice,
            g.sold_on AS date_sold,
            store.item_sold('SHIPPED') AS state,
            COALESCE(r.date, checks.created_at) AS realization_date,
            ROUND(a.price * (COALESCE(rnds.value, cnds.value, 20) / (100 + COALESCE(rnds.value, cnds.value, 20))), 2) AS nds,
            NULL::text AS supplier,
            u.id AS manager
            ,null
            ,null
    --        ,a.in_account_date in_account_date
            ,g.status item_state
            ,a.del adel

        FROM params _
        CROSS JOIN opentech.accounts a
        LEFT JOIN opentech.account_item ai ON ai.account_id = a.id

        LEFT JOIN payments.giftcards g ON a.i = g.id_order_item
        LEFT JOIN opentech.users u ON lower(u.login) = lower(a.user_login)
        LEFT JOIN orders.realization_items_order_items_bindings bi ON bi.id_order_item = a.i
        LEFT JOIN orders.realization_items i ON i.id = bi.id_realization_item
        LEFT JOIN orders.realizations r ON r.id = i.id_realization
        LEFT JOIN payments.order_items_locked_by_check_items il ON il.id_order_item = a.i
        LEFT JOIN payments.check_items ci ON ci.id = il.id_check_item_fullpaid
        LEFT JOIN payments.checks checks ON checks.id = ci.id_check
        LEFT JOIN public.nds rnds ON rnds.id = i.id_nds
        LEFT JOIN public.nds cnds ON cnds.code = ci.vat

        WHERE 
    --    	g.status = ANY(payments.giftcard_status(ARRAY['ISSUED', 'EXPENDED']))
    --        g.sold_on BETWEEN _.frm AND _.til
    --            AND 
                tm_placed BETWEEN _.frm AND _.til
                and ai.id_platform in (7832, 1246, 3252)

    )

    , items AS  (
        SELECT * FROM wares
        UNION ALL SELECT * FROM bundles
        UNION ALL SELECT * FROM services
        UNION ALL SELECT * FROM deliveries
        UNION ALL SELECT * FROM giftcards
    )


    select 

        item.i,
        item.id_platform,
        COALESCE(item.id_order_item, ai.account_id) "ID закаpf",
        item.tm_placed::timestamp,
        item.in_account_date::timestamp,
        item.order_source,
        item.adel

    from items item
    LEFT JOIN opentech.account_item ai ON ai.account_id = item.id_order

    --where 
    --    COALESCE(item.id_order_item, ai.account_id) in (7076147)


'''


## Путь куда будут выгружены файлы¶

In [27]:
public_link = '''https://nextcloud.e2e4.ru/apps/files/?dir=/Public/Отчеты%20(выгрузка%20jupyter%20notebooks)&fileid=2018892'''
AmazonS3_folder_Public = r'/Public/Отчеты (выгрузка jupyter notebooks)'
AmazonS3_path_to_file = r'/ДЛЯ ТЕСТОВ/458/'
#     AmazonS3_file_name = fr'''458 30.01.2023 13.02.2023.xlsx ({getpass.getuser()}-{datetime.datetime.now().strftime("%H:%M:%S")}) {DATE_START_replce} - {DATE_END_replce}.xlsx'''
AmazonS3_file_name = fr'''458 30.01.2023 13.02.2023.xlsx ({getpass.getuser()}-{datetime.datetime.now().strftime("%H:%M:%S")}).xlsx'''
AmazonS3_full_path = AmazonS3_folder_Public + AmazonS3_path_to_file + AmazonS3_file_name
#     print('AmazonS3_file_name, AmazonS3_full_path', AmazonS3_file_name, AmazonS3_full_path)
Amazon_owncloud = owncloud.Client('https://nextcloud.e2e4.ru')
Amazon_owncloud.login(loginAtlassianjhubAdmin, passwordAtlassianjhubAdmin)
# Amazon_owncloud.mkdir('testdir')
AmazonS3_full_path

'/Public/Отчеты (выгрузка jupyter notebooks)/ДЛЯ ТЕСТОВ/458/458 30.01.2023 13.02.2023.xlsx (slebedev-18:25:28).xlsx'

## коннектимся, подставляем даты, сохраняем в файл

In [28]:


#     print( f'''Выбранные даты: {DATE_START_replce.strftime("%d.%m.%Y")}    {DATE_END_replce.strftime("%d.%m.%Y")}
#                ({DATE_START_replce})  ({DATE_END_replce}) \n\nИдет выгрузка с БД немного подождите...\n''' )
#   connect
with psycopg2.connect(dbname=database, user=user, password=password, host=host) as cnxn:
    df_ = pd.read_sql_query(sql_ \
#                              .replace('''DATE_START_replce''', f'''{ DATE_START_replce }''') \
#                              .replace('''DATE_END_replce''', f'''{ DATE_END_replce }''') \
#                              .replace('''BREND_replce''', f'''{ df_1_val }''') \
                         , cnxn) 


print('Выгрузка завершена! \nИдет запись в файл/облако')


sheet_name0 = 'Итоговая'
#     sheet_name1 = 'Продано Прибыль Наценка'
#     sheet_name2 = 'Скомплектовано'
#     sheet_name3 = 'Остаток'
sheets0 = df_
#     sheets1 = df_det.astype({'frm': 'str'}).set_index(df_det.columns[0]).loc[:spl1].reset_index()
#     sheets2 = df_det.astype({'frm': 'str'}).set_index(df_det.columns[0]).loc[spl1:spl2].reset_index().iloc[1:]
#     sheets3 = df_det.astype({'frm': 'str'}).set_index(df_det.columns[0]).loc[spl2:].reset_index().iloc[1:]

excel_sheets = {
    f'1 - {sheet_name0}' : sheets0 
#         ,f'2 - {sheet_name1}' : sheets1 
#         ,f'3 - {sheet_name2}' : sheets2 
#         ,f'4 - {sheet_name3}' : sheets3 
    }
excel_writer = pd.ExcelWriter( temp_tmp + AmazonS3_file_name, engine='xlsxwriter' )

if df_.shape[0] != 0:
    for sheet_name in excel_sheets.keys():
        excel_sheets[sheet_name].iloc[:,:].to_excel(excel_writer, sheet_name=sheet_name, index=False)
        print(
f'''\
{ colored( str(eval(sheet_name[0])) + 'й лист в excel:', 'red')}\n\
{colored(sheet_name, 'green')} \nболее детальная информация в файле по пути:\n\
{colored(AmazonS3_full_path, 'blue', attrs=["underline","bold"])} \n\
!! Идет запись в файл пожалуйста подождите  \n\
''')
    excel_writer.save()
else:
    False 

print('Запись в файл завершена!\n\nИдет запись в облако\n\n') #

try:
    Amazon_owncloud.put_file( 
#           полный путь и там и там!!! 
        AmazonS3_full_path, temp_tmp + AmazonS3_file_name )  
    print( f'Файл записан в облако\n{AmazonS3_full_path} ' )
except:
    print( f'Возникли проблемы при записи в облако{AmazonS3_full_path}'   )

os.remove(temp_tmp + AmazonS3_file_name)
display(HTML(fr"""<a href="{public_link}">Ссылка на nextcloud.e2e4.ru</a>"""))


Выгрузка завершена! 
Идет запись в файл/облако
1й лист в excel:
1 - Итоговая 
более детальная информация в файле по пути:
/Public/Отчеты (выгрузка jupyter notebooks)/ДЛЯ ТЕСТОВ/458/458 30.01.2023 13.02.2023.xlsx (slebedev-18:25:28).xlsx 
!! Идет запись в файл пожалуйста подождите  

Запись в файл завершена!

Идет запись в облако


Файл записан в облако
/Public/Отчеты (выгрузка jupyter notebooks)/ДЛЯ ТЕСТОВ/458/458 30.01.2023 13.02.2023.xlsx (slebedev-18:25:28).xlsx 


HTML(value='<a href="https://nextcloud.e2e4.ru/apps/files/?dir=/Public/Отчеты%20(выгрузка%20jupyter%20notebook…